In [12]:
import torch
import torchvision
import glob
import cv2
import PIL
import os
import matplotlib.pyplot as plt
from unet import UNet
import imutils
import pytorch_ssim
import tqdm
from IPython.display import HTML,display,clear_output
from torch.utils.tensorboard import SummaryWriter
import timm
import datetime
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/fashion_mnist_experiment_1')

In [13]:
class dataset_faces(torch.utils.data.Dataset):
    def __init__(self, fileName, transform_main=None):
        self.fileList = glob.glob(fileName + "*")
        self.transform_main = transform_main

    def __len__(self):
        output = len(self.fileList)
        return output

    def __getitem__(self, idx):
        img = cv2.imread(self.fileList[idx])
        try:
            img = self.transform_main(img)
        except:
            print(self.fileList[idx])

        return self.linedraw(img), img

    def linedraw(self, x):
        transform = torchvision.transforms.Grayscale(3)
        x = transform(x)
        # 3x3カーネルで膨張1回（膨張はMaxPoolと同じ）
        dilated = torch.max_pool2d(x, kernel_size=3, stride=1, padding=1)
        # 膨張の前後でL1の差分を取る
        diff = torch.abs(x - dilated)
        # ネガポジ反転
        x = 1.0 - diff
        return x


def resize_img(img):
    """
    画像をpaddingしながら256x256にする
    """
    height, width, _ = img.shape  # 画像の縦横サイズを取得
    diffsize = abs(height - width)
    padding_half = int(diffsize / 2)

    # 縦長画像→幅を拡張する
    if height > width:
        padding_img = cv2.copyMakeBorder(
            img, 0, 0, padding_half, height - (width + padding_half), cv2.BORDER_CONSTANT, (255, 255, 255)
        )
    # 横長画像→高さを拡張する
    elif width > height:
        padding_img = cv2.copyMakeBorder(
            img, padding_half, width - (height + padding_half), 0, 0, cv2.BORDER_CONSTANT, (255, 255, 255)
        )
    else:
        padding_img = img
    # 最後にリサイズ
    return imutils.resize(padding_img, width=64)


def cvfunc(img):
    img = img[:, :, ::-1]
    img = resize_img(img)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    return PIL.Image.fromarray(img)


def show_tensor(input_image_tensor, f):
    writer.add_image(tag="img",img_tensor=input_image_tensor,global_step=f)
    #img = input_image_tensor.to("cpu").detach().numpy().transpose(1, 2, 0)
    
    # img = img.astype(np.uint8)[0,0,:,:]
    #plt.imshow(img)
    #plt.savefig(f"res\\test_{f}.png")
    # plt.show()
def show_tensor2(input_image_tensor):
    #writer.add_image(tag="img",img_tensor=input_image_tensor,global_step=f)
    img = input_image_tensor.to("cpu").detach().numpy().transpose(1, 2, 0)
    plt.imshow(img)
    #plt.savefig(f"res\\test_{f}.png")
    # plt.show()


In [18]:

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Lambda(cvfunc),
        torchvision.transforms.RandomRotation(degrees=90, fill=(255, 255, 255)),
        torchvision.transforms.RandomHorizontalFlip(p=0.5),
        torchvision.transforms.RandomVerticalFlip(p=0.5),
        #テンソル化はできるだけ最後のほうがいい
        torchvision.transforms.ToTensor(),
    ]
)

linedataset = dataset_faces("image\\", transform)
print(len(linedataset))
train_dataset, val_dataset = torch.utils.data.random_split(linedataset, [int(len(linedataset)*0.8), len(linedataset)-int(len(linedataset)*0.8)])

#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
#val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True)
# モデル定義
model = UNet(3, 3)
# デバイスモデル設定
#
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 損失関数,分類問題のためクロスエントロピー損失関数を利用
# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
#criterion = torch.nn.L1Loss()
criterion =pytorch_ssim.SSIM(window_size = 11)

from timm.scheduler import CosineLRScheduler
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = CosineLRScheduler(optimizer, t_initial=100, lr_min=1e-6, 
                                  warmup_t=3, warmup_lr_init=1e-6, warmup_prefix=True)
EPOCHS = 10000
#a, b = next(iter(train_loader))
#writer.add_graph(model, a)
#print(a.shape, b.shape)

os.makedirs("res", exist_ok=True)

0


In [16]:
if 0:
    writer.add_image(tag="img/l",img_tensor=a[0],global_step=0)
    writer.add_image(tag="img/v",img_tensor=b[0],global_step=0)
    show_tensor2(torchvision.utils.make_grid(torch.cat([a, b])))

In [35]:
now = datetime.datetime.now()
today = now.strftime('%Y年%m月%d日(%A) %H:%M:%S')

display(HTML(f"""
<h1>～機械学習 学習前レポート～</h1>
    <h3>{today}</h3><table>
	<tbody>
		<tr>
<td colspan="2">学習全体レポート</td>
		</tr>
		<tr>
			<td>デバイス名</td>
			<td>{device}</td>
		</tr>
		<tr>
			<td>学習ネットワーク名</td>
			<td>{model.__class__.__name__}</td>
		</tr>
		<tr>
			<td>損失関数</td>
			<td>{criterion.__class__.__name__}</td>
		</tr>
		<tr>
			<td>最適化関数</td>
			<td>{optimizer.__class__.__name__}  学習率 :{optimizer.defaults["lr"]}</td>
		</tr>
        <tr>
			<td>スケジューラ</td>
			<td>{scheduler.__class__.__name__}</td>
		</tr>
		<tr>
			<td>総エポック数</td>
			<td>{EPOCHS}</td>
		</tr>
        
		<tr>
			<td>tensorBoardパス</td>
			<td>{writer.log_dir}</td>
		</tr>
	</tbody>
</table>"""))

In [17]:
def fit(model,device,criterion,optimizer,scheduler,EPOCHS,writer):
    writer.add_text(tag="discription",text_string=f"# Total Epoch : {EPOCHS}  \nNet name{model.__class__.__name__}")
    for epoch in range(EPOCHS):
        display(HTML(f"<h1>Now EPOCH : {epoch}/{EPOCHS}</h1><br>"))
        model.train()  # モデルを学習モードにしてGPUに転送（重要）
        model.to(device)
        trainloss=0
        valiloss=0

        for batch in tqdm.auto.tqdm(train_loader,desc="train"):
            optimizer.zero_grad()  # 必須
            # image ,label = batch #(batch_size, channel, size, size)
            image, label = batch

            image = image.to(device)
            label = label.to(device)  # dtype=torch.long

            preds = model(image)  # (batch_size, num_class)
            # print(preds.dtype)
            loss = 1-criterion(preds.to("cpu", dtype=torch.float), label.to("cpu", dtype=torch.float))  # 必須
            loss.backward()  # 必須
            optimizer.step()  # 必須
            trainloss=trainloss+loss.item()
        scheduler.step(epoch)
        
        #writer.add_scalar("train loss",trainloss/len(train_loader),epoch)
        model.eval()  # 評価モードにする

        with torch.no_grad():  # 必須
            for batch in tqdm.auto.tqdm(val_loader,desc="valid"):
                image, label = batch  # (batch_size, channel, size, size)

                image = image.to(device)
                label = label.to(device)
                preds = model(image)

                loss = 1-criterion(preds.to("cpu", dtype=torch.float), label.to("cpu", dtype=torch.float))  # 必須
                valiloss=valiloss+loss.item()
            writer.add_scalars("loss",{"val_loss":valiloss/len(val_loader),"train_loss":trainloss/len(train_loader)},epoch)

        #if epoch % 10 == 0:
        show_tensor(torchvision.utils.make_grid(torch.cat([image, label, preds])), epoch)
        torch.save(model.state_dict(), "model.pth")
            #exit()
        clear_output(wait=True)

In [18]:
fit(model,device,criterion,optimizer,scheduler,EPOCHS,writer)

train:   0%|          | 0/156 [00:00<?, ?it/s]

KeyboardInterrupt: 